<a href="https://colab.research.google.com/github/2517518603/WebUI-Colab/blob/master/Stable_diffusion_%E9%82%80%E6%9C%88%E6%B5%AE%E7%99%BD%E6%95%B4%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StableDiffusion

https://github.com/AUTOMATIC1111/stable-diffusion-webui

https://github.com/TheLastBen/fast-stable-diffusion

****

此版本使用了WebUi程序，主程序自带许多插件，包含了LoRA插件，并优化了Colab脚本界面

>如果出现报错等问题但是可以正常运行与跑图，就当没有报错吧（本人对python也是一知半解）

****
**更新日志**
>2023/04/16 暂时移除将stable-diffusion主程序放置到云盘的选项（快速链接会出问题）

>2023/04/13 汉化包改为必装插件，使用自制config，新安装的小伙伴默认使用汉化,省去汉化步骤

>2023/04/12 可选插件中，追加了chat-GPT填充标签插件

>2023/04/12 可选插件中，追加了civitai-helper插件

>2023/04/12 可选插件中，修复了Latent Couple/two-shot插件，追加了Composable Lora插件

>2023/04/07 torch更新为2.0.0+cu118 +xformers0.18

>2023/04/07 将模板风格改为自动保存到云盘，删除附加步骤
****

****
**附加下载**

colab跳转百度云盘有些问题，如果点击跳转链接显示页面不存在的话，请复制链接手动打开
1. LatentCoupleHelper工具(永久链接)

  https://pan.baidu.com/s/1psZp9c5AsOZVkLHDxaANoA?pwd=1i72
****


b站个人空间：https://space.bilibili.com/45083971

****
使用教程链接：https://www.bilibili.com/video/BV19N411A7d9/?share_source=copy_web&vd_source=d617bc353483a19771d3b4ffa6416355

白嫖模型库教程：https://www.bilibili.com/video/BV11V4y1Z7Et/?share_source=copy_web&vd_source=d617bc353483a19771d3b4ffa6416355

#1.授权Google云盘并检查GPU(无特殊需求请不要变更参数)

In [ ]:
from google.colab import drive
import os, subprocess

##@markdown 安装运行环境
use_python3_10 = False 
##@markdown 修改为torch-2.0.0+cu118 xformers0.18
use_cu118=True
##@param {type:"boolean"}
# 开始挂载云盘
#@markdown 配置云盘中模型所在文件夹名称
Drive_Name='SD' #@param {type:"string"}
#@markdown 配置colab中stable-diffusion-webui所在文件夹名称
SD_path='stable-diffusion' #@param {type:"string"}
##@markdown 将stable-diffusion主程序也放入云盘（云盘免费用户慎用）
User_netdrive=False 
##@param {type:"boolean"}

net_drive = '/content/gdrive';
if not os.path.exists(net_drive):
  drive.mount(net_drive)
net_base_path = net_drive + "/MyDrive/" + Drive_Name

!mkdir -p {net_base_path}

sd_dir="/content/"
if User_netdrive:
  sd_dir=net_base_path + '/'

webui_dir=sd_dir + SD_path

net_model_path = net_base_path + "/models"
net_embeddings_path = net_base_path + "/embeddings"
net_output_path = net_base_path + "/outputs"
net_lora_path = net_base_path + "/lora"

!mkdir -p {net_model_path}
!mkdir -p {net_model_path}/Stable-diffusion
!mkdir -p {net_model_path}/VAE
!mkdir -p {net_model_path}/Lora
!mkdir -p {net_lora_path}
!mkdir -p {net_output_path}
!mkdir -p {net_embeddings_path}

GPU_m4000 = False
move_path = False
try:
    subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE)
    gpuinfo = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    if 'M4000' in gpuinfo:
        print("注意: 你正在使用的是 Quadro M4000 GPU，它将无法使用 xformers。")
        GPU_m4000 = True
    else:
        print("当前使用："+gpuinfo.strip())
except:
    print("没有GPU可使用，请检查你的运行时类型")
    exit()

%store GPU_m4000


# 2.下载WebUi主程序及必要插件，并替换模型和图片输出路径到云盘
（如果没有将stable-diffusion主程序放入云盘的话，每次重新连接服务器都需要运行）

<s>ps.首次使用请到webui界面的setting -> user interface->拉到最下方的下拉选项 选择本地化插件</s>

汉化包改为必装插件，使用自制config，新安装的小伙伴默认使用汉化

In [ ]:
#@title 主程序和个人推荐插件必装，其他可手动勾选安装，如有特殊需求，可点开代码修改
import os, subprocess
from glob import glob
from pathlib import Path
import sys

exists_sd = False
print("检查stable-diffusion文件夹是否存在:" + webui_dir)
if os.path.exists(webui_dir):
  print("stable-diffusion已存在")
  exists_sd = True
else:
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui {webui_dir}
  # runtime n
  !wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O {webui_dir}/scripts/run_n_times.py
  # 安装webui的deforum插件
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {webui_dir}/extensions/deforum-for-automatic1111-webui
  # 图片浏览插件
  !git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {webui_dir}/extensions/stable-diffusion-webui-images-browser
  # additional插件（lora插件）
  !git clone https://github.com/kohya-ss/sd-webui-additional-networks {webui_dir}/extensions/sd-webui-additional-networks
  # 运行--cloudflared参数分享url的必要插件
  !git clone https://github.com/camenduru/sd-webui-tunnels {webui_dir}/extensions/sd-webui-tunnels
  # controlnet
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git {webui_dir}/extensions/sd-webui-controlnet
  # 汉化插件 首次使用请到webui界面的setting -> user interface->拉到最下方的下拉选项 选择汉化插件
  !rm -r {webui_dir}/localizations
  !git clone https://github.com/dnciyun/Zh-localization.git {webui_dir}/localizations

configpath=''  
#NovelAi参数设置，并替换到云盘保存
if not User_netdrive:
  configpath=net_base_path
else:
  configpath=webui_dir

if not os.path.exists(configpath + "/config.json"):
  %cd {configpath}
  #!wget https://raw.githubusercontent.com/PtO2/stablediffusion_webui_configs/main/config.json
  !wget https://beehomefile.oss-cn-beijing.aliyuncs.com/20210114/config.json

if not os.path.exists(configpath + "/ui-config.json"): 
  %cd {configpath}
  #!wget https://raw.githubusercontent.com/PtO2/stablediffusion_webui_configs/main/ui-config.json
  !wget https://beehomefile.oss-cn-beijing.aliyuncs.com/20210114/ui-config.json
  
!echo NovelAi参数设置成功
!echo 下载WebUi主程序及必要插件成功

#@markdown ---
#@markdown <font size="5" color="#000000">可选插件下载</font> 
#@markdown ---

#@markdown waifu标签（推荐安装）
waifu_tag=True #@param {type:"boolean"}
#@markdown 标签补全（推荐安装）
tagcomplete=True #@param {type:"boolean"}
#@markdown civitai直连插件（直接在civitai下载模型，无需打开civitai）
civitai=False #@param {type:"boolean"}
#@markdown civitai-helper插件（同步模型预览图，也可以在civitai下载模型，但是需要手动输入civitai模型的url）
civitai_helper=False #@param {type:"boolean"}
#@markdown poseX--姿势编辑器(个人推荐这个)
PoseX=True #@param {type:"boolean"}
#@markdown openpose--姿势编辑器
Openpose=False #@param {type:"boolean"}
#@markdown Latent Couple/two-shot插件 & Composable Lora插件 （这两个插件需要配合使用，所以合并下载）
couple=False #@param {type:"boolean"}
#@markdown chat-GPT填充标签插件（初次使用需要在设置中填入OpenAI中申请的API-Key）
chatGpt=False #@param {type:"boolean"}
#@markdown depth lib--手部深度图库
hand_depth_lib=True #@param {type:"boolean"}
#@markdown mov2mov-视频转换插件
mov2mov=False #@param {type:"boolean"}

%cd /content/
plug_c=webui_dir + "/extensions"
if tagcomplete and (not os.path.exists(plug_c+"/a1111-sd-webui-tagcomplete")):
  # tag自动补全插件（推荐安装）
  !git clone https://jihulab.com/hunter0725/a1111-sd-webui-tagcomplete {webui_dir}/extensions/a1111-sd-webui-tagcomplete
  !rm -f {webui_dir}/extensions/a1111-sd-webui-tagcomplete/tags/danbooru.csv
  !wget https://beehomefile.oss-cn-beijing.aliyuncs.com/20210114/danbooru.csv -O {webui_dir}/extensions/a1111-sd-webui-tagcomplete/tags/danbooru.csv

if waifu_tag and (not os.path.exists(plug_c+"/stable-diffusion-webui-wd14-tagger")):
  # waifu标签（推荐安装）
  !git clone https://jihulab.com/hunter0725/stable-diffusion-webui-wd14-tagger {webui_dir}/extensions/stable-diffusion-webui-wd14-tagger

if civitai and (not os.path.exists(plug_c+"/sd-civitai-browser")):
  # civitai直连插件（可不安装）
  !git clone https://github.com/camenduru/sd-civitai-browser {webui_dir}/extensions/sd-civitai-browser

if civitai_helper and (not os.path.exists(plug_c+"/Stable-Diffusion-Webui-Civitai-Helper")):
  # civitai-helper插件（可不安装）
  !git clone https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper {webui_dir}/extensions/Stable-Diffusion-Webui-Civitai-Helper

  

if hand_depth_lib and (not os.path.exists(plug_c+"/sd-webui-depth-lib")):
  !git clone https://github.com/jexom/sd-webui-depth-lib.git {plug_c}/sd-webui-depth-lib

if Openpose and (not os.path.exists(plug_c+"/openpose-editor")):
  !git clone https://github.com/fkunn1326/openpose-editor.git {plug_c}/openpose-editor

if couple:
  if (not os.path.exists(plug_c+"/stable-diffusion-webui-composable-lora")):
   !git clone https://github.com/opparco/stable-diffusion-webui-composable-lora.git {plug_c}/stable-diffusion-webui-composable-lora
  if (not os.path.exists(plug_c+"/stable-diffusion-webui-two-shot")):
   !git clone https://github.com/opparco/stable-diffusion-webui-two-shot {plug_c}/stable-diffusion-webui-two-shot
if PoseX and (not os.path.exists(plug_c+"/posex")):
  !git clone https://github.com/hnmr293/posex.git {plug_c}/posex

if chatGpt and (not os.path.exists(plug_c+"/stable-diffusion-webui-chatgpt-utilities")):
  !git clone https://github.com/hallatore/stable-diffusion-webui-chatgpt-utilities.git {plug_c}/stable-diffusion-webui-chatgpt-utilities


if mov2mov:
  #Mov2mov
  modnet_webcam = '1Nf1ZxeJZJL8Qx9KadcYYyEmmlKhTADxX'
  modnet_photographic = '1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz'


  %cd {plug_c}
  if not os.path.exists(plug_c+"/sd-webui-mov2mov"):
      !git clone https://github.com/Scholar01/sd-webui-mov2mov.git
  else:
      %cd sd-webui-mov2mov
      !git pull

  all_cn = [modnet_webcam, modnet_photographic]

  cndir = plug_c + "/sd-webui-mov2mov/models"
  !mkdir -p {cndir}
  %cd $cndir
  for lnk in all_cn:
      !gdown --fuzzy $lnk

!echo 可选插件下载完成
#@markdown ---
#@markdown <font size="5" color="#294595">ControlNet模型下载</font> 
#@markdown ---
#@markdown canny（边缘）
canny=True #@param {type:"boolean"}
#@markdown depth（深度）
depth=True #@param {type:"boolean"}
#@markdown hed（Hed边缘）
hed=True #@param {type:"boolean"}
#@markdown mlsd（线段）
mlsd=True #@param {type:"boolean"}
#@markdown scribble（手绘）
scribble=True #@param {type:"boolean"}
#@markdown openpose（骨骼）
openpose=True #@param {type:"boolean"}

!mkdir -p {plug_c}/sd-webui-controlnet/models
%cd {plug_c}/sd-webui-controlnet/models

if canny and (not os.path.exists(plug_c+"/sd-webui-controlnet/models/control_canny-fp16.safetensors")):
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors

if depth and (not os.path.exists(plug_c+"/sd-webui-controlnet/models/control_depth-fp16.safetensors")):
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors

if hed and (not os.path.exists(plug_c+"/sd-webui-controlnet/models/control_hed-fp16.safetensors")):
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors

if mlsd and (not os.path.exists(plug_c+"/sd-webui-controlnet/models/control_mlsd-fp16.safetensors")):
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors
  
if scribble and (not os.path.exists(plug_c+"/sd-webui-controlnet/models/control_scribble-fp16.safetensors")):
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors

if openpose and (not os.path.exists(plug_c+"/sd-webui-controlnet/models/control_openpose-fp16.safetensors")):
  !wget https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors
  
!echo ControlNet模型下载完成

!echo 开始路径替换工作

#把stable-diffusion原模型路径修改，创建云盘保存模型路径链接
if not User_netdrive:
  !rm -f {webui_dir}/config.json
  !ln -s {net_base_path}/config.json {webui_dir}/config.json
  !rm -f {webui_dir}/ui-config.json
  !ln -s {net_base_path}/ui-config.json {webui_dir}/ui-config.json
if not exists_sd:
  !echo 建立云盘模型路径链接
  !mv {webui_dir}/models {webui_dir}/models_bak
  !ln -s {net_model_path} {webui_dir}/models 
  for root, dirs, files in os.walk(webui_dir + "/models_bak"):
    for dir in dirs:
      tmpPath =os.path.join(root, dir)
      if (not os.path.exists(webui_dir+"/models/"+dir)):
        !cp -r {tmpPath} {webui_dir}/models/{dir}
  !echo 建立云盘图片输出路径链接
  !ln -s {net_output_path} {webui_dir}/outputs
  !echo 建立云盘Lora模型路径链接
  if not os.path.exists(webui_dir + "/extensions/sd-webui-additional-networks/models/lora/lora_plus"):    
    !ln -s {net_lora_path} {webui_dir}/extensions/sd-webui-additional-networks/models/lora/lora_plus
    !echo 建立Lora模型到sd-webui-additional-networks路径的链接
  if not os.path.exists(webui_dir + "/models/Lora/lora_plus"):
    !ln -s {net_lora_path} {webui_dir}/models/Lora/lora_plus
  !echo 建立云盘embeddings模型路径链接
  !ln -s {net_embeddings_path} {webui_dir}/embeddings/embeddings_plus
  if os.path.exists(net_base_path + '/styles.csv'):    
    !cp {net_base_path}/styles.csv {webui_dir}/styles.csv
      
!echo 链接替换完成
move_path = True

!echo 全工程完成


# 3.下载模型（已下载的不会重复执行下载）
主模型较大，请根据云盘容量酌情下载
如果云盘中已有模型，可以跳过这部

In [ ]:
import os,sys,time
from IPython.utils import capture
from IPython.display import clear_output
from IPython.display import HTML
from subprocess import getoutput
from google.colab import drive
#@markdown <font size="5" color="#dd0000">主模型下载</font>
#@markdown ---

#@markdown anything-v4.5 （7.3G左右）
anything = False #@param {type:"boolean"}
#@markdown Counterfeit-V2.5
counterfeit = False #@param {type:"boolean"}
#@markdown AbyssOrangeMix2 （7.3G左右）
abyssMix2 = False #@param {type:"boolean"}
#@markdown AbyssOrangeMix3
AbyssMix3 = False #@param {type:"boolean"}
#@markdown pastel-mix
pastel = False #@param {type:"boolean"}
#@markdown chilloutmix_NiPruned （7.3G左右）
chilloutmix = True #@param {type:"boolean"}

!mkdir -p {net_model_path}/Stable-diffusion/
!mkdir -p {net_model_path}/VAE/
if anything:
  if not os.path.exists(net_model_path + "/Stable-diffusion/anything-v4.5.ckpt"):
    %cd {net_model_path}/Stable-diffusion/
    !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5.ckpt
  if not os.path.exists(net_model_path + "/Stable-diffusion/anything-v4.5.vae.pt"):
    %cd {net_model_path}/Stable-diffusion/
    !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt

if counterfeit:
  if not os.path.exists(net_model_path + "/Stable-diffusion/Counterfeit-V2.5.safetensors"):
    %cd {net_model_path}/Stable-diffusion/
    !wget https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.safetensors
  if not os.path.exists(net_model_path + "/Stable-diffusion/Counterfeit-V2.5.vae.pt"):
    %cd {net_model_path}/VAE/
    !wget https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt

if abyssMix2:
  if not os.path.exists(net_model_path + "/Stable-diffusion/AbyssOrangeMix2_sfw.ckpt"):
    %cd {net_model_path}/Stable-diffusion/
    !wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_sfw.ckpt
  if not os.path.exists(net_model_path + "/VAE/orangemix.vae.pt"):
    %cd {net_model_path}/VAE/
    !wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt

if AbyssMix3:
  if not os.path.exists(net_model_path + "/Stable-diffusion/AOM3A1.safetensors"):
    %cd {net_model_path}/Stable-diffusion/
    !wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1.safetensors
  if not os.path.exists(net_model_path + "/VAE/orangemix.vae.pt"):
    %cd {net_model_path}/VAE/
    !wget https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt

if pastel:
  if not os.path.exists(net_model_path + "/Stable-diffusion/pastelmix-better-vae.safetensors"):
    %cd {net_model_path}/Stable-diffusion/
    !wget https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-better-vae.safetensors

if chilloutmix:
  if not os.path.exists(net_model_path + "/Stable-diffusion/Chilloutmix.safetensors"):
    %cd {net_model_path}/Stable-diffusion/
    !wget https://huggingface.co/swl-models/chilloutmix/resolve/main/Chilloutmix.safetensors
  if not os.path.exists(net_model_path + "/VAE/vae-ft-mse-840000-ema-pruned.safetensors"):
    %cd {net_model_path}/VAE/
    !wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
!echo 主模型下载完成

#@markdown ---
#@markdown <font size="5" color="#22dd00">Lora模型下载</font>
#@markdown ---
#@markdown 韩国人偶（真人）
korean  = True #@param {type:"boolean"}

#@markdown 台湾人偶（真人）
taiwan = True #@param {type:"boolean"}
#裸化（适用真人）
nudify = False 
#y道（适用真人）
vaginas = False 
if korean:
  if not os.path.exists(net_lora_path + "/Korean-doll-likeness.safetensors"):
    %cd {net_lora_path}
    !wget https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Korean-doll-likeness.safetensors

if taiwan:
  if not os.path.exists(net_lora_path + "/Taiwan-doll-likeness.safetensors"):
    %cd {net_lora_path}
    !wget https://huggingface.co/AnonPerson/ChilloutMix/resolve/main/Taiwan-doll-likeness.safetensors

if nudify:
  if not os.path.exists(net_lora_path + "/nudify.safetensors"):
    %cd {net_lora_path}
    !wget https://civitai.com/api/download/models/12041 -O {net_lora_path}/nudify.safetensors




if vaginas:
  if not os.path.exists(net_lora_path + "/vaginas.safetensors"):
    %cd {net_lora_path}
    !wget https://civitai.com/api/download/models/12318 -O {net_lora_path}/vaginas.safetensors
!echo Lora模型下载完成

#@markdown ---
#@markdown <font size="5" color="#2222dd">Embedding下载</font>
#@markdown ---
#@markdown deep-negative-v1x（深度否定）
deep = True #@param {type:"boolean"}
#@markdown EasyNegative（画质优化）
Easy = True #@param {type:"boolean"}
#@markdown 修手（手部优化）
hand = True #@param {type:"boolean"}
#@markdown Ulzzang（真人模型优化）
ulzzang = True #@param {type:"boolean"}

if Easy:
  if not os.path.exists(net_embeddings_path + "/embeddings/EasyNegative.pt"):
    %cd {net_embeddings_path}
    !wget https://github.com/tuzh6/model/raw/controlnet/EasyNegative.pt


if deep:
  if not os.path.exists(net_embeddings_path + "/embeddings/deep-negative-v1x.pt"):
    %cd {net_embeddings_path}
    !wget https://civitai.com/api/download/models/5637 -O {net_model_path}/embeddings/deep-negative-v1x.pt



if hand:
  if not os.path.exists(net_embeddings_path + "/embeddings/bad_prompt_version2.pt"):
    %cd {net_embeddings_path}
    !wget https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt_version2.pt

if ulzzang:
  if not os.path.exists(net_embeddings_path + "/embeddings/ulzzang-6500.pt"):
    %cd {net_embeddings_path}
    !wget https://huggingface.co/yesyeahvh/ulzzang-6500/resolve/main/ulzzang-6500.pt

!echo Embedding模型下载完成



%cd /content/






#4.运行主程序
忙了半天不就是为了它吗？

点击运行，开始愉快的使用stable-diffusion画图吧

In [ ]:
from IPython.utils import capture
from subprocess import getoutput
import os,sys,time,fileinput,base64,sys,glob,threading,shutil,torch

def watch_and_copy():
  file_path = webui_dir + "/styles.csv"
  mtime = time.time()
  time_str = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(mtime))

  # 打印时间字符串
  #print(time_str)
  #print(file_path)
  
  while True:
    #print(os.path.exists(file_path))
    if os.path.exists(file_path):
      # 获取文件修改时间
      fileTime = os.path.getmtime(file_path)
      time_str = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(fileTime))

      # 打印时间字符串
      #print(time_str)
      # 如果文件修改时间在当前时间之后
      if mtime < fileTime:
          time_str = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(mtime))
          # 打印时间字符串
          print(time_str)
          # 拷贝文件到目标文件夹中
          !rm -f {net_base_path}/styles.csv
          !cp {webui_dir}/styles.csv {net_base_path}/styles.csv
          mtime = time.time()
    time.sleep(5)

# 创建线程并启动
if not User_netdrive:
  t = threading.Thread(target=watch_and_copy)
  t.setDaemon(True) # 将线程设置为后台线程，即主线程退出时该线程也会退出
  t.start()

#@markdown 是否启用登录账号和密码
Use_account_pwd = False #@param {type:"boolean"}
user_acount='account' #@param {type:"string"}
user_password='password' #@param {type:"string"}
#@markdown 跳过计算模型hashi值
No_hashing=False #@param {type:"boolean"}
#@markdown 启用API
Enable_API=False #@param {type:"boolean"}
#@markdown 加载VAE文件：
Use_VAE=True #@param {type:"boolean"}
#@markdown 启用xformers
Enable_Xformers=True #@param {type:"boolean"}
#@markdown 其他自定义启动参数(可添加上面没有列出的参数，多个参数中间请添加空格)
User_Cmd_Args=' ' #@param {type:"string"}

#切换到python3.10
if use_python3_10:
  !wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
  !bash ./py310.sh -b -f -p /usr/local
  !python -m ipykernel install --name "py310" --user
  use_python3_10 = False

now_use_cu118 = torch.__version__=="2.0.0+cu118"
if now_use_cu118:
  print("已使用torch2.0.0+cu118")
  !pip install -U xformers
else:
  !pip uninstall torch -y
  !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
  !pip install -U xformers

if not os.path.exists(webui_dir):
     print('主程序不存在，请先下载主程序！')
     raise SystemExit

extra_cmd=''
if User_netdrive:
  %cd {webui_dir}/extensions/sd-webui-controlnet/
  !git pull
  %cd {webui_dir}
  !git pull
  !git reset --hard HEAD
  !sed -i '/Checking out commit for/d' {webui_dir}/launch.py
  !git checkout -- ./modules/*.py
else:  
  %cd {webui_dir}

if any([os.path.exists(os.path.join(webui_dir+"/models/Stable-diffusion", f)) for f in os.listdir(os.path.join(webui_dir+"/models/Stable-diffusion")) if f.endswith(('.ckpt', '.safetensors'))]):
  !sed -i 's@weight_load_location =.*@weight_load_location = "cuda"@' $webui_dir/modules/shared.py
  !sed -i "s@os.path.splitext(model_file)@os.path.splitext(model_file); map_location='cuda'@" $webui_dir/modules/sd_models.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" $webui_dir/modules/extras.py
    
print("本整合包所使用的程序均来自互联网，本人只完成了整合，本整合使用完全免费 --By B站 邀月浮白")

if Use_VAE:
    ckpt_files = glob.glob(sd_dir + '/*.ckpt')
    safetensors_files = glob.glob(sd_dir + '/*.safetensors')
    pt_files = glob.glob(sd_dir + '/*.pt')
    if len(ckpt_files) > 0:
        extra_cmd = extra_cmd + " --vae-path " + ckpt_files[0] + " --no-half-vae"
    elif len(safetensors_files) > 0:
        extra_cmd = extra_cmd + " --vae-path " + safetensors_files[0] + " --no-half-vae"
    elif len(pt_files) > 0:
        extra_cmd = extra_cmd + " --vae-path " + pt_files[0] + " --no-half-vae"

if No_hashing:
  extra_cmd += ' --no-hashing '
if Enable_API:
  extra_cmd += ' --api '

extra_cmd=extra_cmd+' '+User_Cmd_Args + ' '

if Use_account_pwd:
  extra_cmd = '--gradio-auth '+ user_acount+':'+ user_password + ' ' + extra_cmd

!python launch.py --share --disable-safe-unpickle --enable-insecure-extension-access --cloudflared $extra_cmd {'' if GPU_m4000 else '--xformers'}